<a href="https://colab.research.google.com/github/NimitBajaj/Semantic-Genre-Classification-of-Comedians/blob/main/NLP_in_Python_5_(Topic_Modeling).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic Modeling

## Introduction

Another popular text analysis technique is called topic modeling. The ultimate goal of topic modeling is to find various topics that are present in your corpus. Each document in the corpus will be made up of at least one topic, if not multiple topics.

In this notebook, we will be covering the steps on how to do **Latent Dirichlet Allocation (LDA)**, which is one of many topic modeling techniques. It was specifically designed for text data.

To use a topic modeling technique, you need to provide (1) a document-term matrix and (2) the number of topics you would like the algorithm to pick up.

Once the topic modeling technique is applied, your job as a human is to interpret the results and see if the mix of words in each topic make sense. If they don't make sense, you can try changing up the number of topics, the terms in the document-term matrix, model parameters, or even try a different model.

## Topic Modeling - Attempt #1 (All Text)

In [ ]:
# Let's read in our document-term matrix
import pandas as pd
import pickle

data = pd.read_pickle('data_clean.pkl')
data

,transcript,full_name
ali,ladies gentlemen please welcome stage ali w...,Ali Wong
anthony,thank you thank you thank you san francisco th...,Anthony Jeselnik
ari,every town i figured out every city gets eithe...,Ari Shaffir
bill,all right thank you thank you very much thank ...,Bill Burr
billm,bill mahers anyone else seeing this sharp u...,Bill Mahar
bo,bo burnham inside comedy special written dir...,Bo Burnham
dave,this dave he tells dirty jokes for living th...,Dave Chappelle
fluffy,gabriel iglesias legend fluffy lively celeb...,Gabriel Iglesias
hasan,san jose whats upnice see you whats happening...,Hasan Minhaj
jim,ladies gentlemen please welcome stage mr ji...,Jim Jefferies


In [ ]:
!pip install gensim

In [ ]:
import scipy
print(scipy.__version__)


1.13.1


In [ ]:
!pip install scipy==1.11.4

In [ ]:
# Import the necessary modules for LDA with gensim
# Terminal / Anaconda Navigator: conda install -c conda-forge gensim
from gensim import matutils, models
import scipy.sparse

# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
# One of the required inputs is a term-document matrix
tdm = data.transpose()
tdm.head()

,ali,anthony,bill,brian,dave,david,demetri,hannibal,jerry,jim,joe,john,kyle,louis,mike,patton,pete,richard,ricky,rory
aaaaah,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
aaah,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
aah,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0
abandoned,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
abc,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)
print(corpus)

In [ ]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())
id2word

{212: 'ali',
 10700: 'wong',
 613: 'baby',
 1797: 'cobra',
 9901: 'transcript',
 8274: 'scraps',
 5520: 'loft',
 8700: 'skip',
 2008: 'content',
 6145: 'moviesmovie',
 7945: 'reviewsmovie',
 9906: 'transcriptstv',
 8408: 'seriestv',
 9904: 'transcriptscomedystandup',
 1858: 'comedy',
 9905: 'transcriptsgeorge',
 1395: 'carlindave',
 1514: 'chappelleinterviewsplayboy',
 4844: 'interviewsmusichistorybooksopinions',
 8309: 'search',
 8310: 'searchcomedy',
 8403: 'september',
 10702: 'wongs',
 9047: 'stand',
 8934: 'special',
 2475: 'delves',
 8438: 'sexual',
 119: 'adventures',
 4426: 'hoarding',
 8041: 'rocky',
 8019: 'road',
 7326: 'pregnancy',
 3406: 'feminism',
 9542: 'terrible',
 5260: 'ladies',
 3870: 'gentlemen',
 10532: 'welcome',
 9035: 'stage',
 10701: 'wonghi',
 4327: 'hello',
 9575: 'thank',
 1869: 'coming',
 8510: 'shit',
 6923: 'pee',
 5998: 'minutes',
 3167: 'everybody',
 1870: 'comingum',
 3211: 'exciting',
 2373: 'day',
 10822: 'year',
 10016: 'turned',
 10837: 'yes',
 40

Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term), we need to specify two other parameters - the number of topics and the number of passes. Let's start the number of topics at 2, see if the results make sense, and increase the number from there.

In [ ]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.006*"shit" + 0.005*"said" + 0.005*"want" + 0.005*"going" + 0.004*"fucking" + 0.004*"man" + 0.004*"life" + 0.004*"thing" + 0.004*"good" + 0.004*"guy"'),
 (1,
  '0.007*"said" + 0.004*"good" + 0.004*"shit" + 0.004*"did" + 0.004*"thing" + 0.004*"really" + 0.004*"fucking" + 0.004*"little" + 0.004*"want" + 0.004*"fuck"')]

In [ ]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.007*"said" + 0.005*"shit" + 0.005*"really" + 0.004*"man" + 0.004*"way" + 0.004*"going" + 0.004*"good" + 0.004*"mean" + 0.004*"thing" + 0.004*"okay"'),
 (1,
  '0.006*"fucking" + 0.006*"said" + 0.006*"shit" + 0.005*"want" + 0.005*"fuck" + 0.005*"good" + 0.005*"going" + 0.004*"thing" + 0.004*"went" + 0.004*"little"'),
 (2,
  '0.004*"life" + 0.003*"phone" + 0.003*"great" + 0.003*"gotta" + 0.003*"wanna" + 0.003*"love" + 0.003*"little" + 0.003*"things" + 0.002*"thing" + 0.002*"lot"')]

In [ ]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

[(0,
  '0.005*"want" + 0.004*"real" + 0.004*"did" + 0.003*"thing" + 0.003*"look" + 0.003*"good" + 0.003*"kyle" + 0.003*"ill" + 0.003*"okay" + 0.002*"doctor"'),
 (1,
  '0.005*"okay" + 0.005*"thing" + 0.004*"really" + 0.004*"ill" + 0.004*"life" + 0.004*"mean" + 0.004*"come" + 0.004*"going" + 0.004*"good" + 0.004*"love"'),
 (2,
  '0.005*"said" + 0.005*"fucking" + 0.004*"went" + 0.004*"little" + 0.004*"id" + 0.004*"day" + 0.004*"shit" + 0.003*"ive" + 0.003*"going" + 0.003*"god"'),
 (3,
  '0.008*"said" + 0.006*"shit" + 0.005*"good" + 0.005*"fucking" + 0.005*"want" + 0.005*"going" + 0.004*"man" + 0.004*"fuck" + 0.004*"guy" + 0.004*"thing"')]

In [ ]:
# LDA for num_topics = 5
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=5, passes=10)
lda.print_topics()

[(0,
  '0.008*"said" + 0.007*"fucking" + 0.005*"going" + 0.005*"want" + 0.005*"shit" + 0.005*"fuck" + 0.005*"day" + 0.004*"went" + 0.004*"thing" + 0.004*"did"'),
 (1,
  '0.000*"shit" + 0.000*"really" + 0.000*"said" + 0.000*"did" + 0.000*"come" + 0.000*"love" + 0.000*"want" + 0.000*"going" + 0.000*"good" + 0.000*"man"'),
 (2,
  '0.007*"shit" + 0.006*"good" + 0.006*"man" + 0.005*"said" + 0.005*"life" + 0.005*"want" + 0.004*"thing" + 0.004*"come" + 0.004*"did" + 0.004*"little"'),
 (3,
  '0.007*"fcking" + 0.006*"fck" + 0.005*"shit" + 0.005*"really" + 0.005*"mean" + 0.004*"going" + 0.004*"okay" + 0.004*"thing" + 0.004*"kid" + 0.004*"man"'),
 (4,
  '0.005*"said" + 0.005*"okay" + 0.004*"hey" + 0.004*"clinton" + 0.004*"little" + 0.004*"way" + 0.004*"really" + 0.004*"day" + 0.003*"look" + 0.003*"went"')]

In [ ]:
# LDA for num_topics = 6
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=6, passes=10)
lda.print_topics()

[(0,
  '0.006*"going" + 0.006*"crime" + 0.006*"want" + 0.005*"said" + 0.003*"thank" + 0.003*"come" + 0.003*"ticket" + 0.003*"little" + 0.003*"uh" + 0.003*"make"'),
 (1,
  '0.015*"said" + 0.008*"says" + 0.006*"jenny" + 0.006*"goes" + 0.005*"id" + 0.005*"really" + 0.005*"mean" + 0.005*"point" + 0.004*"kind" + 0.004*"uh"'),
 (2,
  '0.007*"said" + 0.007*"shit" + 0.006*"fuck" + 0.006*"fucking" + 0.006*"good" + 0.005*"man" + 0.005*"want" + 0.005*"did" + 0.004*"going" + 0.004*"thing"'),
 (3,
  '0.007*"shit" + 0.004*"okay" + 0.004*"little" + 0.004*"going" + 0.004*"really" + 0.004*"fucking" + 0.004*"thing" + 0.004*"want" + 0.004*"man" + 0.004*"good"'),
 (4,
  '0.009*"life" + 0.007*"thing" + 0.005*"shit" + 0.005*"good" + 0.005*"tit" + 0.004*"really" + 0.004*"course" + 0.004*"guy" + 0.004*"kids" + 0.004*"murder"'),
 (5,
  '0.007*"said" + 0.006*"fucking" + 0.006*"little" + 0.005*"went" + 0.005*"id" + 0.004*"thing" + 0.004*"going" + 0.004*"ive" + 0.004*"life" + 0.004*"joke"')]

These topics aren't looking too great. We've tried modifying our parameters. Let's try modifying our terms list as well.

## Topic Modeling - Attempt #2 (Nouns Only)

One popular trick is to look only at terms that are from one part of speech (only nouns, only adjectives, etc.). Check out the UPenn tag set: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html.

In [ ]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)]
    return ' '.join(all_nouns)

In [ ]:
# Read in the cleaned data, before the CountVectorizer step
data_clean = pd.read_pickle('data_clean.pkl')
data_clean

,transcript
ali,ali wong baby cobra transcript scraps from...
anthony,anthony jeselnik thoughts and prayers full ...
bill,bill burr im sorry you feel that way full t...
bo,scraps from the loft \t\tskip to content movi...
brian,brian regan on the rocks transcript scraps...
dave,dave chappelle the age of spin transcript ...
david,david cross worst daddy in the world transc...
demetri,demetri martin demetri deconstructed transc...
hannibal,hannibal buress comedy camisado transcript ...
hasan,scraps from the loft \t\tskip to content movi...


In [ ]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\A06BH\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\A06BH\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_eng.zip.


True

In [ ]:
# Apply the nouns function to the transcripts to filter only on nouns
try:
    data_nouns = pd.DataFrame(data_clean.transcript.apply(nouns))
except LookupError as e:
    print(e)


In [ ]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']

# Merge the default English stop words with the additional ones
stop_words = list(text.ENGLISH_STOP_WORDS.union(add_stop_words))

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns.transcript)

# Create a DataFrame
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names_out())
data_dtmn.index = data_nouns.index

data_dtmn


,aah,abc,abcs,ability,abortion,abortions,aboutyes,abovetheground,abraham,absence,...,zippers,zombie,zombies,zone,zones,zoo,zoologist,zoom,zoos,éclair
ali,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
anthony,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bill,0,0,1,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
bo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
brian,0,0,0,0,0,0,0,0,0,0,...,0,0,0,3,1,0,0,0,0,0
dave,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
david,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
demetri,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
hannibal,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
hasan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [ ]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.006*"gon" + 0.006*"guy" + 0.006*"way" + 0.006*"day" + 0.005*"cause" + 0.005*"thing" + 0.004*"hes" + 0.004*"man" + 0.004*"theyre" + 0.004*"shit"'),
 (1,
  '0.010*"thing" + 0.009*"man" + 0.009*"shit" + 0.008*"life" + 0.007*"day" + 0.007*"hes" + 0.006*"gon" + 0.006*"way" + 0.006*"cause" + 0.006*"guy"')]

In [ ]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.009*"thing" + 0.009*"man" + 0.008*"shit" + 0.008*"life" + 0.007*"hes" + 0.007*"day" + 0.007*"guy" + 0.006*"cause" + 0.006*"way" + 0.006*"gon"'),
 (1,
  '0.008*"day" + 0.007*"lot" + 0.007*"joke" + 0.006*"shit" + 0.005*"women" + 0.005*"baby" + 0.005*"school" + 0.004*"husband" + 0.004*"anthony" + 0.004*"mom"'),
 (2,
  '0.008*"gon" + 0.007*"way" + 0.007*"thing" + 0.006*"guy" + 0.005*"cause" + 0.005*"man" + 0.005*"kind" + 0.005*"house" + 0.005*"life" + 0.005*"things"')]

In [ ]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.010*"thing" + 0.009*"day" + 0.009*"shit" + 0.008*"man" + 0.007*"hes" + 0.006*"life" + 0.006*"cause" + 0.006*"way" + 0.006*"gon" + 0.006*"kid"'),
 (1,
  '0.007*"life" + 0.007*"things" + 0.007*"way" + 0.007*"guy" + 0.006*"thing" + 0.006*"house" + 0.006*"man" + 0.005*"cause" + 0.005*"theyre" + 0.005*"kind"'),
 (2,
  '0.009*"gon" + 0.008*"man" + 0.007*"guy" + 0.007*"shit" + 0.005*"thing" + 0.005*"way" + 0.005*"hes" + 0.005*"cause" + 0.005*"fuck" + 0.005*"lot"'),
 (3,
  '0.010*"life" + 0.009*"thing" + 0.007*"cause" + 0.007*"hes" + 0.007*"way" + 0.006*"day" + 0.006*"kids" + 0.005*"clinton" + 0.005*"guy" + 0.005*"kid"')]

## Topic Modeling - Attempt #3 (Nouns and Adjectives)

In [ ]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)]
    return ' '.join(nouns_adj)

In [ ]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.transcript.apply(nouns_adj))
data_nouns_adj

,transcript
ali,ali wong baby cobra transcript scraps loft ski...
anthony,anthony jeselnik thoughts prayers full transcr...
bill,bill im sorry way full transcript scraps loft ...
bo,scraps loft skip moviesmovie reviewsmovie tran...
brian,brian regan rocks scraps loft skip moviesmovie...
dave,chappelle age spin transcript scraps loft skip...
david,david cross worst daddy world transcript scrap...
demetri,demetri martin demetri transcript scraps loft ...
hannibal,hannibal buress comedy camisado transcript scr...
hasan,scraps loft skip moviesmovie reviewsmovie tran...


In [ ]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names_out())
data_dtmna.index = data_nouns_adj.index
data_dtmna

,aaaaah,aah,abandoned,abc,abcs,ability,abject,able,ablebodied,aboriginal,...,zippers,zombie,zombies,zone,zones,zoo,zoologist,zoom,zoos,éclair
ali,0,0,0,1,0,0,0,2,0,0,...,0,1,0,0,0,0,0,0,0,0
anthony,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bill,1,0,0,0,1,0,0,1,0,0,...,0,1,1,0,0,0,0,0,0,0
bo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
brian,0,0,0,0,0,0,0,3,0,0,...,0,0,0,3,1,0,0,0,0,0
dave,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
david,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
demetri,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
hannibal,0,0,0,0,0,0,0,9,0,0,...,1,0,0,0,0,0,0,0,0,0
hasan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [ ]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.005*"shit" + 0.003*"joke" + 0.003*"okay" + 0.003*"shes" + 0.003*"women" + 0.003*"phone" + 0.003*"sure" + 0.003*"car" + 0.003*"kids" + 0.003*"room"'),
 (1,
  '0.010*"shit" + 0.006*"fuck" + 0.004*"kid" + 0.004*"kids" + 0.004*"dude" + 0.003*"women" + 0.003*"shes" + 0.003*"fucking" + 0.003*"ta" + 0.003*"joke"')]

In [ ]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.012*"shit" + 0.005*"kid" + 0.005*"fuck" + 0.004*"dude" + 0.004*"kids" + 0.004*"shes" + 0.004*"women" + 0.003*"fucking" + 0.003*"ta" + 0.003*"okay"'),
 (1,
  '0.009*"fuck" + 0.007*"shit" + 0.005*"fucking" + 0.005*"women" + 0.004*"room" + 0.004*"ahah" + 0.003*"sure" + 0.003*"men" + 0.003*"son" + 0.003*"crime"'),
 (2,
  '0.005*"joke" + 0.004*"okay" + 0.004*"shit" + 0.004*"shes" + 0.004*"car" + 0.004*"kids" + 0.004*"phone" + 0.003*"ta" + 0.003*"fck" + 0.003*"kid"')]

In [ ]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.013*"shit" + 0.004*"dude" + 0.004*"women" + 0.004*"fck" + 0.004*"fuck" + 0.004*"kids" + 0.003*"different" + 0.003*"ta" + 0.003*"ahah" + 0.003*"shes"'),
 (1,
  '0.007*"fuck" + 0.005*"joke" + 0.004*"shit" + 0.004*"shes" + 0.004*"ha" + 0.003*"fucking" + 0.003*"pubes" + 0.003*"sure" + 0.003*"ta" + 0.003*"jokes"'),
 (2,
  '0.008*"shit" + 0.006*"kid" + 0.006*"fuck" + 0.006*"fucking" + 0.005*"women" + 0.005*"kids" + 0.004*"shes" + 0.004*"dude" + 0.003*"ta" + 0.003*"dog"'),
 (3,
  '0.005*"joke" + 0.004*"okay" + 0.004*"school" + 0.004*"sure" + 0.004*"mom" + 0.003*"friend" + 0.003*"parents" + 0.003*"dad" + 0.003*"crime" + 0.003*"jenny"')]

## Identify Topics in Each Document

Out of the 9 topic models we looked at, the nouns and adjectives, 4 topic one made the most sense. So let's pull that down here and run it through some more iterations to get more fine-tuned topics.

In [ ]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=80)
ldana.print_topics()

[(0,
  '0.014*"shit" + 0.007*"fuck" + 0.005*"kids" + 0.004*"women" + 0.004*"dude" + 0.004*"kid" + 0.004*"fucking" + 0.004*"ta" + 0.004*"shes" + 0.003*"different"'),
 (1,
  '0.006*"joke" + 0.004*"kid" + 0.004*"shes" + 0.003*"ta" + 0.003*"okay" + 0.003*"fuck" + 0.003*"phone" + 0.003*"church" + 0.003*"hell" + 0.003*"fucking"'),
 (2,
  '0.005*"jenny" + 0.004*"car" + 0.003*"sure" + 0.003*"friend" + 0.003*"accident" + 0.003*"sex" + 0.003*"phone" + 0.002*"women" + 0.002*"idea" + 0.002*"room"'),
 (3,
  '0.004*"place" + 0.004*"doctor" + 0.004*"tour" + 0.003*"sandwich" + 0.003*"kyle" + 0.003*"carry" + 0.002*"shit" + 0.002*"shoes" + 0.002*"gay" + 0.002*"guns"')]

These four topics look pretty decent. Let's settle on these for now.
* Topic 0: Profanity
* Topic 1: Joking
* Topic 2: Relationships
* Topic 3: Doctors & Travel

In [ ]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
list(zip([max(doc, key=lambda x: x[1])[0] for doc in corpus_transformed], data_dtmna.index))


[(0, 'ali'),
 (1, 'anthony'),
 (0, 'bill'),
 (2, 'bo'),
 (1, 'brian'),
 (0, 'dave'),
 (1, 'david'),
 (0, 'demetri'),
 (0, 'hannibal'),
 (2, 'hasan'),
 (1, 'jerry'),
 (0, 'jim'),
 (0, 'joe'),
 (0, 'john'),
 (3, 'kyle'),
 (0, 'louis'),
 (2, 'mike'),
 (1, 'patton'),
 (2, 'pete'),
 (1, 'richard'),
 (1, 'ricky'),
 (0, 'rory')]

For a first pass of LDA, these kind of make sense to me, so we'll call it a day for now.
* Topic 0: Profanity(Ali, bill, dave, demetri,hannibal, jim, joe,john, louis, rory)
* Topic 1: Joking(anthony, brian, david, jerry, patton, richard, ricky)
* Topic 2: Relationships(bo,hasan,mike,pete)
* Topic 3: Doctors & Travel(kyle)

### Assignment:
1. Try further modifying the parameters of the topic models above and see if you can get better topics.
2. Create a new topic model that includes terms from a different [part of speech](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) and see if you can get better topics.

### Noun and Adverb

In [ ]:
def adverb(text):
    '''Given a string of text, tokenize the text and pull out only the adverbs.'''
    is_adverb = lambda pos: pos[:2] == 'NN' or pos[:2] == 'RB'  # 'RB' is the POS tag for adverbs
    tokenized = word_tokenize(text)
    all_adverbs = [word for (word, pos) in pos_tag(tokenized) if is_adverb(pos)]
    return ' '.join(all_adverbs)


In [ ]:
data_adv = pd.DataFrame(data_clean.transcript.apply(adverb))
data_adv

,transcript
ali,ali baby cobra transcript scraps skip reviewsm...
anthony,anthony jeselnik thoughts prayers transcript s...
bill,bill sorry way transcript scraps skip reviewsm...
bo,scraps skip reviewsmovie transcriptstv seriest...
brian,regan rocks scraps skip reviewsmovie transcrip...
dave,chappelle age spin transcript scraps skip revi...
david,david cross daddy world transcript scraps skip...
demetri,demetri martin demetri scraps skip reviewsmovi...
hannibal,buress comedy camisado transcript scraps skip ...
hasan,scraps skip reviewsmovie transcriptstv seriest...


In [ ]:
cvadv = CountVectorizer(stop_words=stop_words, max_df=0.8)
data_cvadv = cvadv.fit_transform(data_adv.transcript)
data_dtmadv = pd.DataFrame(data_cvadv.toarray(), columns=cvadv.get_feature_names_out())
data_dtmadv.index = data_adv.index
data_dtmadv


,aah,abc,abcs,ability,abortion,abortions,aboutyes,abovetheground,abraham,abroad,...,zippers,zombie,zombies,zone,zones,zoo,zoologist,zoom,zoos,éclair
ali,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
anthony,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bill,0,0,1,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
bo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
brian,0,0,0,0,0,0,0,0,0,0,...,0,0,0,3,1,0,0,0,0,0
dave,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
david,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
demetri,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
hannibal,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
hasan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Create the gensim corpus
corpusadv = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmadv.transpose()))

# Create the vocabulary dictionary
id2wordadv = dict((v, k) for k, v in cvadv.vocabulary_.items())

In [ ]:
# Let's start with 2 topics
ldaadv = models.LdaModel(corpus=corpusadv, num_topics=2, id2word=id2wordadv, passes=10)
ldaadv.print_topics()

[(0,
  '0.008*"shit" + 0.005*"women" + 0.004*"shes" + 0.004*"fuck" + 0.004*"kid" + 0.003*"men" + 0.003*"fck" + 0.003*"school" + 0.003*"okay" + 0.003*"god"'),
 (1,
  '0.008*"shit" + 0.006*"joke" + 0.006*"baby" + 0.005*"kids" + 0.004*"fuck" + 0.004*"kid" + 0.004*"shes" + 0.004*"god" + 0.003*"car" + 0.003*"hell"')]

In [ ]:
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.006*"fuck" + 0.006*"shit" + 0.004*"sure" + 0.003*"friend" + 0.003*"car" + 0.003*"room" + 0.003*"ahah" + 0.003*"women" + 0.003*"shes" + 0.002*"wife"'),
 (1,
  '0.012*"shit" + 0.005*"kid" + 0.005*"dude" + 0.005*"kids" + 0.005*"ta" + 0.004*"mom" + 0.003*"dad" + 0.003*"fck" + 0.003*"fuck" + 0.003*"joke"'),
 (2,
  '0.005*"shit" + 0.004*"shes" + 0.004*"kid" + 0.004*"fucking" + 0.004*"women" + 0.004*"fuck" + 0.004*"joke" + 0.004*"different" + 0.004*"kids" + 0.004*"okay"')]

In [ ]:
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.008*"shit" + 0.007*"fuck" + 0.005*"joke" + 0.005*"ahah" + 0.004*"anthony" + 0.004*"women" + 0.003*"gay" + 0.003*"place" + 0.003*"fucking" + 0.003*"okay"'),
 (1,
  '0.002*"cool" + 0.002*"women" + 0.002*"saturday" + 0.001*"dude" + 0.001*"em" + 0.001*"ha" + 0.001*"live" + 0.001*"drunk" + 0.001*"somethin" + 0.001*"miles"'),
 (2,
  '0.007*"shit" + 0.005*"fuck" + 0.004*"kids" + 0.004*"kid" + 0.004*"shes" + 0.004*"fucking" + 0.004*"women" + 0.004*"ta" + 0.003*"car" + 0.003*"sure"'),
 (3,
  '0.010*"shit" + 0.005*"kid" + 0.005*"kids" + 0.003*"shes" + 0.003*"church" + 0.003*"different" + 0.003*"hannibal" + 0.003*"jesus" + 0.003*"fuck" + 0.003*"okay"')]


* Topic 0: Profanity
* Topic 1: Casual Socializing
* Topic 2: Relationships
* Topic 3: Religion

In [ ]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
list(zip([max(doc, key=lambda x: x[1])[0] for doc in corpus_transformed], data_dtmna.index))


[(2, 'ali'),
 (0, 'anthony'),
 (2, 'bill'),
 (1, 'bo'),
 (2, 'brian'),
 (0, 'dave'),
 (3, 'david'),
 (2, 'demetri'),
 (3, 'hannibal'),
 (1, 'hasan'),
 (2, 'jerry'),
 (2, 'jim'),
 (2, 'joe'),
 (2, 'john'),
 (0, 'kyle'),
 (3, 'louis'),
 (2, 'mike'),
 (0, 'patton'),
 (3, 'pete'),
 (1, 'richard'),
 (2, 'ricky'),
 (2, 'rory')]


* Topic 0: Profanity(Anthony,Dave, Kyle, Patton)
* Topic 1: Casual Socializing(Bo, Hasan, Richard)
* Topic 2: Relationships (Ali,Bill, Brian, Demetri, Jerry, Jim ,Joe, John, Mike, Ricky, Rory)
* Topic 3: Religion(David, Hannibal, Louis, Pete)

### NUMBER OF PASSES TRIAL

In [ ]:
for i in range(10,50):
    print(f"--- Pass {i} ---")
    ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=i)
    print(ldana.print_topics())

--- Pass 10 ---
[(0, '0.005*"fuck" + 0.004*"shit" + 0.004*"shes" + 0.003*"wife" + 0.003*"ta" + 0.003*"phone" + 0.003*"friend" + 0.003*"dog" + 0.003*"sure" + 0.003*"place"'), (1, '0.013*"shit" + 0.006*"fuck" + 0.005*"women" + 0.005*"dude" + 0.004*"fucking" + 0.004*"kids" + 0.004*"joke" + 0.004*"kid" + 0.003*"okay" + 0.003*"ta"'), (2, '0.004*"kid" + 0.004*"shes" + 0.003*"kids" + 0.003*"church" + 0.003*"shit" + 0.003*"joke" + 0.003*"different" + 0.003*"crime" + 0.003*"women" + 0.003*"course"')]
--- Pass 11 ---
[(0, '0.012*"shit" + 0.006*"kids" + 0.005*"kid" + 0.005*"dude" + 0.005*"joke" + 0.004*"shes" + 0.004*"women" + 0.004*"ta" + 0.004*"fuck" + 0.003*"mom"'), (1, '0.004*"phone" + 0.004*"car" + 0.004*"okay" + 0.004*"sure" + 0.003*"ta" + 0.003*"hell" + 0.003*"jenny" + 0.003*"shes" + 0.003*"sex" + 0.003*"friend"'), (2, '0.007*"fuck" + 0.007*"shit" + 0.005*"fucking" + 0.004*"women" + 0.003*"kid" + 0.003*"room" + 0.003*"sure" + 0.003*"shes" + 0.003*"ahah" + 0.003*"church"')]
--- Pass 12 ---


In [ ]:
from gensim import models

# Initialize variables to track topics
previous_first_two_words = []
flag=False

for i in range(10, 50):
    print(f"--- Pass {i} ---")

    # Train the LDA model with the current pass number
    ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=i)

    # Get the topics for this pass
    topics = ldana.print_topics()

    # Extract the first two words from each topic
    current_first_two_words = [topic[1].split(" + ")[:2] for topic in topics]

    # Compare the current topics' first two words with previous ones
    for topic in current_first_two_words:
        if topic in previous_first_two_words:
            print(f"--- Topics are repeating at pass {i} ---")
            flag=True
            break
    else:
            # If topics don't repeat, store the current first two words
            previous_first_two_words.extend(current_first_two_words)
            print(ldana.print_topics())
    if flag:
         break


--- Pass 10 ---
[(0, '0.005*"shit" + 0.004*"course" + 0.004*"cool" + 0.004*"kids" + 0.004*"women" + 0.003*"joke" + 0.003*"okay" + 0.003*"tit" + 0.003*"weird" + 0.003*"ha"'), (1, '0.010*"shit" + 0.005*"women" + 0.004*"fuck" + 0.003*"fucking" + 0.003*"friend" + 0.003*"shes" + 0.003*"room" + 0.003*"kid" + 0.003*"sure" + 0.003*"men"'), (2, '0.009*"shit" + 0.008*"fuck" + 0.004*"ta" + 0.004*"fucking" + 0.003*"dude" + 0.003*"car" + 0.003*"hell" + 0.003*"work" + 0.003*"place" + 0.003*"kids"'), (3, '0.007*"joke" + 0.006*"kid" + 0.005*"shit" + 0.005*"fck" + 0.005*"kids" + 0.005*"shes" + 0.004*"different" + 0.004*"church" + 0.004*"okay" + 0.004*"school"')]
--- Pass 11 ---
[(0, '0.004*"shit" + 0.004*"kid" + 0.004*"okay" + 0.003*"church" + 0.003*"kids" + 0.003*"shes" + 0.003*"different" + 0.003*"crime" + 0.003*"jesus" + 0.003*"weird"'), (1, '0.011*"shit" + 0.005*"fuck" + 0.005*"kids" + 0.004*"kid" + 0.004*"shes" + 0.004*"dude" + 0.004*"women" + 0.004*"fucking" + 0.003*"ta" + 0.003*"car"'), (2, '0.0